<a href="https://colab.research.google.com/github/AndrewTrefilov/solution-participants-AITrain/blob/master/Detection_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center">Detection Baseline</h1>

![](https://habrastorage.org/webt/mi/ca/_c/mica_cm-rdj8z3rrztfk4a-k3_q.png)

---

> **ЗАДАЧА:** На основе размеченных фотографий необходимо создать алгоритм детекции следующих объектов: `'Car', 'Human', 'Wagon', 'FacingSwitchL', 'FacingSwitchR', 'FacingSwitchNV', 'TrailingSwitchL', 'TrailingSwitchR', 'TrailingSwitchNV', 'SignalE', 'SignalF'`


---

<h1 align="center">Оглавление</h1>

- [1. Train/val/test split](#part1) <br>
- [2. Подготовим датасет](#part2) <br>
    - [2.1 Организация директорий](#part2.1) <br>
    - [2.2 Создадим файл с описанием датасетов](#part2.2) <br>
    - [2.3 Подготовим изображения и разметку](#part2.3) <br>
- [3. Выбор модели](#part3) <br>
- [4. Обучение модели](#part4) <br>
- [5. Тестирование модели](#part5) <br>
- [6. Оценивание](#part5) <br>

---

<h1 align="center">0. Установка и импорт необходимых библиотек</h1>

> **NOTE**: В качестве бейзлайна выбрана модель yolov5, обучать модель будем через командную строку

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git > /dev/null
!cd yolov5 && pip install -r requirements.txt > /dev/null

In [1]:
import os
import yaml
from PIL import Image
import random
import numpy as np
from shutil import copyfile
import json
from tqdm import tqdm
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/gdrive')


# https://github.com/AndrewTrefilov/solution-participants-AITrain
from detection.helpers import makedirs, get_yolo_labels, copy_images

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


<a id='part1'></a>
<h1 align="center">1. Train/val/test split</h1>

In [2]:
# !cp ../gdrive/MyDrive/AIJ-2021/input/AITrain_train.zip ./
# !unzip -q AITrain_train.zip

In [3]:
PATH_TO_BBOXES = 'train_data/bboxes'
PATH_TO_IMAGES = 'train_data/images'
PATH_TO_SAVE_LABELS = 'train_data/labels'

CLASS_NAMES = ['Car', 'Human', 'Wagon', 'FacingSwitchL', 'FacingSwitchR', 'FacingSwitchNV', 'TrailingSwitchL', 'TrailingSwitchR', 'TrailingSwitchNV', 'SignalE', 'SignalF']

In [4]:
random.seed(0)
IMAGES_NAME = os.listdir(PATH_TO_IMAGES)
random.shuffle(IMAGES_NAME)

train_images = IMAGES_NAME[:int(len(IMAGES_NAME)*0.7)]
val_images = IMAGES_NAME[int(len(IMAGES_NAME)*0.7): int(len(IMAGES_NAME)*0.85)]
test_images = IMAGES_NAME[int(len(IMAGES_NAME)*0.85):]
len(train_images), len(val_images), len(test_images)

(5742, 1230, 1231)

In [5]:
DATASET_TYPE2IMAGE = {'train': train_images,
                      'val': val_images,
                      'test': test_images}

IMAGE2DATASET_TYPE = {}
for key, values in DATASET_TYPE2IMAGE.items():
    for file_name in values:
        IMAGE2DATASET_TYPE[file_name] = key

<a id='part2'></a>
<h1 align="center">2. Подготовим датасет</h1>

<a id='part2.1'></a>
<h2 align="center">2.1 Организация директорий</h2>


Данные - изображения и разметка для **train**, **val** и **test** выборок, должны быть организованы в соответствии с приведенным ниже примером.  
YOLOv5 автоматически находит разметку для каждого изображения, заменяя директорию */ images /* в каждом пути изображения на */ labels /*. 
Например:  
`train_data/images/train/0.jpg  # image`  
`train_data/labels/train/0.txt  # label`  
   
---
  
*(Поэтому в файле с описанием датасетов достаточно указать только путь до директорий с изображениями, например для **train** выборки `train_data/images/train/`)*


```
train_data
├── images
│   ├── test
│   ├── train
│   └── val
└── labels
    ├── test
    ├── train
    └── val
```

<a id='part2.2'></a>
<h2 align="center">2.2 Создадим файл с описанием датасетов</h2>


```
# Пути до датасетов (до директорий с изображениями)
train: ../train_data/images/train/
val: ../train_data/images/val/
test: ../train_data/images/test/

# Количество классов
nc: 11

# Названия классов
names: ['Car', 'Human', 'Wagon', 'FacingSwitchL', 'FacingSwitchR', 'FacingSwitchNV', 'TrailingSwitchL', 'TrailingSwitchR', 'TrailingSwitchNV', 'SignalE', 'SignalF']
```

In [6]:
aitrain_dataset = ["train: ../train_data/images/train/" + "\n",
                 "val: ../train_data/images/val/" + "\n",
                 "test: ../train_data/images/test/" + "\n\n",
                 "nc: 11" + "\n\n",
                 "names: ['Car', 'Human', 'Wagon', 'FacingSwitchL', 'FacingSwitchR', 'FacingSwitchNV', 'TrailingSwitchL', 'TrailingSwitchR', 'TrailingSwitchNV', 'SignalE', 'SignalF']",
                ]

with open(r'yolov5/data/aitrain_dataset.yaml', 'w') as f:
    f.writelines(aitrain_dataset)

<a id='part2.3'></a>
<h2 align="center">2.3 Подготовим изображения и разметку</h2>

In [7]:
# Создадим нужные директории
makedirs(PATH_TO_SAVE_LABELS, PATH_TO_IMAGES, DATASET_TYPE2IMAGE)

In [8]:
# Создадим и сохраним разметку
get_yolo_labels(PATH_TO_BBOXES, PATH_TO_SAVE_LABELS, PATH_TO_IMAGES, IMAGE2DATASET_TYPE, CLASS_NAMES)

In [9]:
# Скопируем изображения
copy_images(IMAGES_NAME, IMAGE2DATASET_TYPE, PATH_TO_IMAGES)

100%|██████████| 8203/8203 [03:52<00:00, 35.31it/s]


<a id='part3'></a>
<h1 align="center">3. Выбор модели</h1>

Выберите предварительно обученную модель, с которой можно начать обучение.
Предобученные веса загружаются автоматически.
Подробнее по [ссылке](https://github.com/ultralytics/yolov5#pretrained-checkpoints)
![](https://habrastorage.org/webt/qk/as/y_/qkasy_mlt0kkcih7d7z2wk6li6g.png)

<a id='part4'></a>
<h1 align="center">4. Обучение модели</h1>

По пути `yolov5/data/hyps/hyp.scratch.yaml` - находится файл с настройками гиперпараметров.

Поскольку у нас в задаче присутствуют классы, зависящие от левой и правой стороны, поставим вероятность для аугментации fliplr равной нулю:
`fliplr: 0.0  # image flip left-right (probability)` , создадим новый файл - `yolov5/data/hyps/hyp_aitrain.yaml`

In [10]:
with open("yolov5/data/hyps/hyp.scratch.yaml", "r") as f:
    hyps = yaml.safe_load(f)

hyps['fliplr'] = 0.0

with open("yolov5/data/hyps/hyp_aitrain.yaml", 'w') as f:
    yaml.dump(hyps, f)

In [1]:
 # Train YOLOv5m6 on train_dataset for 30 epochs
!cd yolov5 && python train.py --img 1280 --batch 8 --epochs 30 --data aitrain_dataset.yaml --weights yolov5m6.pt --hyp data/hyps/hyp_aitrain.yaml --name exp6

train: weights=yolov5m6.pt, cfg=, data=aitrain_dataset.yaml, hyp=data/hyps/hyp_aitrain.yaml, epochs=30, batch_size=8, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, entity=None, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-492-gb0ade48 torch 1.9.0+cu102 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

hyperparameters: anchor_t=4.0, box=0.05, cls=0.5, cls_pw=1.0, copy_paste=0.0, degrees=0.0, fl_gamma=0.0, fliplr=0.0, flipud=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, iou_t=0.2, lr0=0.01, lrf=0.2, mixup=0.0, momentum=0.937, mosaic=1.0, obj=1.0, obj_pw=1.0, perspective=0.0, scale=0.

<a id='part5'></a>
<h1 align="center">5. Тестирование модели</h1>

Оценим качество лучших весов модели (которые сохранились по пути `yolov5/runs/train/exp6/weights/best.pt`) на отложенной выборке - `test dataset`

In [3]:
!cd yolov5 && python val.py \
               --img 1280 \
               --data aitrain_dataset.yaml\
               --batch-size 16 \
               --task test \
               --weights runs/train/exp6/weights/best.pt

val: data=/content/yolov5/data/aitrain_dataset.yaml, weights=['runs/train/exp6/weights/best.pt'], batch_size=16, imgsz=1280, conf_thres=0.001, iou_thres=0.6, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False
YOLOv5 🚀 v5.0-492-gb0ade48 torch 1.9.0+cu102 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 396 layers, 35491344 parameters, 0 gradients, 51.5 GFLOPs
test: Scanning '../train_data/labels/test.cache' images and labels... 1231 found, 0 missing, 142 empty, 0 corrupted: 100% 1231/1231 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 77/77 [02:08<00:00,  1.67s/it]
                 all       1231       6847       0.62      0.515      0.518      0.217
                 Car       1231       2230      0.771       0.71      0.764      0.439
               Human   

<a id='part6'></a>
<h1 align="center">6. Оценивание</h1>

> **NOTE**: 
> - Лучшие веса модели, необходимо поместить в директорию `models/` и назвать `detection_model.pt`.
> - Смотрите пример посылки `sample_submission.zip`.
> - Проверяющая система будет принимать в качестве ваших предсказаний detection_predictions.json формата COCO, подробнее по [ссылке](https://github.com/cocodataset/cocoapi/blob/master/results/instances_val2014_fakebbox100_results.json) и на официальном [сайте](https://cocodataset.org/#format-data).

> - Функция `detection.detection_predict.postprocess` - генерирует нужного вида предсказание для изображения.

> - Функция `evaluation.orig_box2coco.make_coco_detection_ann` - конвертирует боксы данные участникам от организаторов к COCO Object Detection формату.